<a href="https://colab.research.google.com/github/jmontano1987/pdg/blob/main/Proyecto_de_Grado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Proyecto de Grado**

**Título:** Modelo para la detección de Noticias Falsas en tweets con formato de texto en el contexto político colombiano para las elecciones del año 2022.

**Integrantes:**
* Jesus Eduardo Flores Q.
* Juan Gabriel Montaño M.

#**Instalaciones**

In [88]:
!pip install tweepy
!pip3 install tqdm
!pip install matplotlib
!pip install wordcloud

#**Librerias**

In [89]:
import io
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np
import string
import tweepy
from tqdm.notebook import tqdm as tqdm_notebook
from dateutil import tz
from datetime import datetime 

#**Funciones**

In [90]:
#Funcion que convierte el formato de la fecha del tweet en hora GMT -5 Colombia
def convert_date(str_date):
  str_format_date = '%Y-%m-%d %H:%M:%S' 
  from_zone = tz.gettz('UTC')
  to_zone = tz.gettz('America/Bogota')
  date_convert = datetime.strptime(str(str_date), str_format_date)
  date_convert = date_convert.replace(tzinfo=from_zone)
  date_convert = date_convert.astimezone(to_zone)
  date_convert = datetime.strftime(date_convert,'%d/%m/%Y %H:%M:%S')
  return date_convert

In [91]:
#Funcion para obtener los tweets de un usuario especifio de twitter
def get_tweets(api,username):
  tweets={}
  for tweet in tqdm_notebook(tweepy.Cursor(api.user_timeline, screen_name = username).items()):
      tweets[tweet.id]= {
          'text': tweet.text,
          'created_at': convert_date(tweet.created_at),
          'author_name': tweet.user.name,
          'author_handler': str(tweet.user.screen_name),
          'user_description': tweet.user.description,
          'user_location': tweet.user.location,
          'user_followers_count': tweet.user.followers_count,
          'user_created_at': convert_date(tweet.user.created_at),
          'user_following_count': tweet.user.friends_count,
          'user_verified': tweet.user.verified,
          'lang': tweet.lang,
          'tweet_hashtags': tweet.entities['hashtags'],
          'mention_names': ["@" + d['screen_name'] for d in tweet.entities['user_mentions'] if 'screen_name' in d],
          'mention_ids': [d['id'] for d in tweet.entities['user_mentions'] if 'id' in d],
          'n_retweets': tweet.retweet_count,
          'n_favorites': tweet.favorite_count,
          'is_retweet': hasattr(tweet, 'retweeted_status')
      }
  df_tweets = pd.DataFrame.from_dict(tweets, orient='index')
  df_tweets['id'] = df_tweets.index
  return df_tweets
    #print(tweet.text)

#**Twitter**

Credenciales Consumo API

In [92]:
# API Keys para autenticacion
api_key = "UcZ9gR5yMV4MlKTFaSwhC1aNX" 
api_secret = "R5BjKcmHan6MGaX98UMhytQfLiZI7TLl9AATGtixJLwmAU2DFR"
access_token = "1452452402070794241-rKitK0leSslMs9XoXEDBTxOwplOGGU"
access_token_secret = "lBAxKokr4Hkg3DxvaUMTWkAqU0E9O5KfZCmJoPulnfSRn"

In [93]:
# Autorización para consumo API
auth = tweepy.OAuthHandler(api_key, api_secret) 
# Acceso al API
auth.set_access_token(access_token, access_token_secret) 
# Instancia para llamar a la API 
api = tweepy.API(auth, wait_on_rate_limit=True) 

In [94]:
#Obtener Tweets de cuenta
df_tweets=get_tweets(api,"juanmontan0")

0it [00:00, ?it/s]

In [95]:
df_tweets.head()

,text,created_at,author_name,author_handler,user_description,user_location,user_followers_count,user_created_at,user_following_count,user_verified,lang,tweet_hashtags,mention_names,mention_ids,n_retweets,n_favorites,is_retweet,id
1488526595614773253,Esto es una prueba para la tesis de grado,01/02/2022 09:56:10,juanmontan0,juanmontan0,,,0,24/10/2021 20:50:17,8,False,es,[],[],[],0,0,False,1488526595614773253
1488522887137046543,prueba 3,01/02/2022 09:41:26,juanmontan0,juanmontan0,,,0,24/10/2021 20:50:17,8,False,es,[],[],[],0,0,False,1488522887137046543
1488522868623355912,prueba 2,01/02/2022 09:41:21,juanmontan0,juanmontan0,,,0,24/10/2021 20:50:17,8,False,es,[],[],[],0,0,False,1488522868623355912
1488521198170513414,prueba,01/02/2022 09:34:43,juanmontan0,juanmontan0,,,0,24/10/2021 20:50:17,8,False,es,[],[],[],0,0,False,1488521198170513414


#**Cargar Corpus**

In [96]:
from google.colab import files
uploaded = files.upload()

Saving Corpus_Noticias_Falsas_Delimitado.csv to Corpus_Noticias_Falsas_Delimitado (1).csv


In [97]:
df = pd.read_csv(io.BytesIO(uploaded['Corpus_Noticias_Falsas_Delimitado.csv']))

In [98]:
print(df.TEXT.iloc[0])

El presidente Iván Duque condenó fuertemente el atentado terrorista que se presentó en la noche de este viernes, 7 de enero, contra miembros del Escuadrón Móvil Antidisturbios (Esmad) en la ciudad de Cali.


In [99]:
df=df.drop(['Nombre','Unnamed: 8','Unnamed: 9'], axis=1)

#**Preprocesamiento**

* Convertir Mayusculas a minusculas
* Eliminar signos de puntuación
* Eliminar tildes

In [100]:
df['headline_Clean']=df['headline'].str.lower().str.translate(str.maketrans('','',string.punctuation)).str.replace("\r", " ").str.translate(str.maketrans('áéíóúüñÁÉÍÓÚÜÑ','aeiouunAEIOUUN'))
df['TEXT_Clean']=df['TEXT'].str.lower().str.translate(str.maketrans('','',string.punctuation)).str.replace("\r", " ").str.translate(str.maketrans('áéíóúüñÁÉÍÓÚÜÑ','aeiouunAEIOUUN'))

In [101]:
print(df.TEXT_Clean.iloc[0])

el presidente ivan duque condeno fuertemente el atentado terrorista que se presento en la noche de este viernes 7 de enero contra miembros del escuadron movil antidisturbios esmad en la ciudad de cali


In [102]:
df.head()

,ID,CATEGORIA,TOPIC,SOURCE,headline,TEXT,LINK,headline_Clean,TEXT_Clean
0,1,Noticia Real,Politico,El tiempo,Duque dice que Eln se adjudicó atentado y busc...,El presidente Iván Duque condenó fuertemente e...,https://www.eltiempo.com/politica/gobierno/eln...,duque dice que eln se adjudico atentado y busc...,el presidente ivan duque condeno fuertemente e...
1,2,Noticia Real,Politico,El espectador,"Gustavo Petro se reunirá con Pedro Sánchez, pr...","Gustavo Petro se reunirá con Pedro Sánchez, pr...",https://www.elespectador.com/politica/?utm_sou...,gustavo petro se reunira con pedro sanchez pre...,gustavo petro se reunira con pedro sanchez pre...
2,3,Noticia Real,Politico,El tiempo,Marta Lucía Ramírez se queja por servicio en e...,Marta Lucía Ramírez se queja por servicio en e...,https://www.eltiempo.com/politica/gobierno/mar...,marta lucia ramirez se queja por servicio en e...,marta lucia ramirez se queja por servicio en e...
3,4,Noticia Real,Politico,El espectador,"Elecciones de 2022 requieren mayor cuidado, po...","Elecciones de 2022 requieren mayor cuidado, po...",https://www.elespectador.com/politica/eleccion...,elecciones de 2022 requieren mayor cuidado por...,elecciones de 2022 requieren mayor cuidado por...
4,5,Noticia Real,Politico,El Nuevo Siglo,Procuraduría citó a mandatarios por fiestas en...,ANTE LA expansión de la variante ómicron en el...,https://www.elnuevosiglo.com.co/articulos/01-0...,procuraduria cito a mandatarios por fiestas en...,ante la expansion de la variante omicron en el...


* Remover stop words

In [103]:
nltk.download('stopwords')
stop_words_esp = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
